In [0]:
from zipfile import ZipFile
import os
from google.colab import files
! pip install tensorboardx

ROOT = '/content'

     |████████████████████████████████| 245kB 30.0MB/s 


Install ngrok for tunneling

In [0]:
if not os.path.exists(os.path.join(ROOT, "ngrok-stable-linux-amd64.zip")):
  !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip ngrok-stable-linux-amd64.zip
  
LOG_DIR = os.path.join(ROOT,'saved/log/')
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

--2019-06-03 09:54:18--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.60.111, 52.45.111.123, 34.206.253.53, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.60.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  6.14MB/s    in 2.6s    

2019-06-03 09:54:21 (6.14 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


Setup tensorboard

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://eeaada9f.ngrok.io


Upload project as .zip file and extract it

In [0]:
file = files.upload()

zip_file = ZipFile(os.path.join(ROOT,list(file.keys())[0]))
zip_file.extractall(ROOT)
zip_file.close()

Saving Archive.zip to Archive.zip


Modify parameters and train model

In [0]:
from config import CONFIG  
from parse_config import ConfigParser
from train import main

CONFIG['arch']['args']['num_channels'] = 64
CONFIG['arch']['args']['num_stacks'] = 4
CONFIG['arch']['args']['num_blocks'] = 7
CONFIG['arch']['args']['kernel_size'] = 5
CONFIG['sigma'] = 20
CONFIG['prediction_blur'] = 20
CONFIG['threshold'] = 0.01
CONFIG['trainer']['epochs'] = 50

main(ConfigParser(CONFIG))

StackedHourglassNet(
  (conv1): Conv2d(1, 64, kernel_size=(1, 3), stride=(2, 3), padding=(8, 38))
  (conv2): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(4, 14))
  (relu): ReLU()
  (hgs): ModuleList(
    (0): Hourglass(
      (pre_bottleneck_blocks): ModuleList(
        (0): Bottleneck(
          (conv1): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU()
        )
        (1): Bottleneck(
          (conv1): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU()
        )
        (2): Bottleneck(
          (conv1): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:804: UserWarning: Using a target size (torch.Size([1, 1, 1507328])) that is different to the input size (torch.Size([1507328])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction)


Train Epoch: 1 [0/1 (0%)] Loss: 0.198230
    epoch          : 1
    loss           : 0.19822952151298523
    percentage_correct_keypoints: 0.0
    keypoint_distance_loss: 112.92026828320708
    val_loss       : 0.12208107113838196
    val_percentage_correct_keypoints: 0.0
    val_keypoint_distance_loss: 111.09701204252042
Train Epoch: 2 [0/1 (0%)] Loss: 0.122081
    epoch          : 2
    loss           : 0.12208107113838196
    percentage_correct_keypoints: 0.0
    keypoint_distance_loss: 111.09701204252042
    val_loss       : 0.04761606827378273
    val_percentage_correct_keypoints: 0.0
    val_keypoint_distance_loss: 112.34595829471917
Train Epoch: 3 [0/1 (0%)] Loss: 0.047616
    epoch          : 3
    loss           : 0.04761606454849243
    percentage_correct_keypoints: 0.0
    keypoint_distance_loss: 112.34595829471917
    val_loss       : 0.02143167331814766
    val_percentage_correct_keypoints: 0.0
    val_keypoint_distance_loss: 103.50218110204702
Train Epoch: 4 [0/1 (0%)] Lo

0.001561046694405377